
## **Langkah 1: Preprocessing Data (Tokenisasi dan Normalisasi)**

### **Proses**:

Kita mulai dengan mempersiapkan dataset dan melakukan **tokenisasi** (mengubah kalimat menjadi kata-kata) serta **normalisasi** (misalnya, semua menjadi lowercase).

In [ ]:
# Dataset
data = [
    ("beli sekarang", "Spam"),
    ("diskon besar", "Spam"),
    ("saya suka belajar", "Ham"),
    ("belajar sekarang di rumah", "Ham")
]

# Tokenisasi dan normalisasi (case folding)
def preprocess(text):
    return text.lower().split()

# Tokenisasi dan normalisasi untuk seluruh dataset
processed_data = [(preprocess(text), label) for text, label in data]
print(processed_data)

[(['beli', 'sekarang'], 'Spam'), (['diskon', 'besar'], 'Spam'), (['saya', 'suka', 'belajar'], 'Ham'), (['belajar', 'sekarang', 'di', 'rumah'], 'Ham')]



## **Langkah 2: Membuat Vocabulary (Kamus Kata Unik)**

### **Proses**:

Kita membuat **vocabulary** yang berisi kata-kata unik yang ada di dataset. Ini akan menjadi fitur untuk representasi teks.

In [ ]:
# Membuat vocabulary
vocab = set(word for text, _ in processed_data for word in text)
vocab = list(vocab)  # Menjadikan vocab sebagai list
print(vocab)

['suka', 'besar', 'beli', 'sekarang', 'rumah', 'di', 'saya', 'diskon', 'belajar']



## **Langkah 3: Representasi Data dengan Bag of Words (BoW)**

### **Proses**:

Kita mengubah setiap kalimat menjadi vektor **Bag of Words (BoW)**, di mana setiap elemen vektor mewakili frekuensi kemunculan kata dalam kalimat berdasarkan vocabulary yang telah dibuat.

In [ ]:
# Fungsi untuk representasi BoW
def text_to_bow(text, vocab):
    return [text.count(word) for word in vocab]

# Menerapkan BoW ke setiap teks dalam dataset
X = [text_to_bow(text, vocab) for text, _ in processed_data]
print(X)

[[0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 0, 0, 1]]



## **Langkah 4: Menghitung Probabilitas Kelas (P(Spam) dan P(Ham))**

### **Proses**:

Hitung probabilitas masing-masing kelas (`Spam` dan `Ham`). Ini adalah **P(Spam)** dan **P(Ham)** yang dibutuhkan dalam rumus Naive Bayes.

In [4]:
# Menghitung probabilitas kelas
from collections import Counter

labels = [label for _, label in processed_data]
label_counts = Counter(labels)

P_spam = label_counts['Spam'] / len(labels)
P_ham = label_counts['Ham'] / len(labels)

print(f"P(Spam) = {P_spam}")
print(f"P(Ham) = {P_ham}")


P(Spam) = 0.5
P(Ham) = 0.5


## **Langkah 5: Menghitung Likelihood (P(w|C)) untuk setiap kata**

### **Proses**:

Untuk setiap kata dalam vocabulary, kita akan menghitung **P(w|Spam)** dan **P(w|Ham)** menggunakan **Laplace smoothing** agar tidak ada nilai probabilitas yang nol.

In [5]:
# Fungsi untuk menghitung P(w|C) menggunakan Laplace Smoothing
def compute_likelihood(class_data, vocab, total_words_in_class, vocab_size):
    word_counts = Counter(word for text in class_data for word in text)
    likelihood = {}
    
    for word in vocab:
        count = word_counts[word] + 1  # Laplace smoothing
        prob = count / (total_words_in_class + vocab_size)
        likelihood[word] = prob
        
    return likelihood

# Pisahkan data berdasarkan kelas
spam_data = [text for text, label in processed_data if label == 'Spam']
ham_data = [text for text, label in processed_data if label == 'Ham']

# Total kata dalam Spam dan Ham
total_spam_words = sum(len(text) for text in spam_data)
total_ham_words = sum(len(text) for text in ham_data)

# Hitung P(w|Spam) dan P(w|Ham)
P_w_given_spam = compute_likelihood(spam_data, vocab, total_spam_words, len(vocab))
P_w_given_ham = compute_likelihood(ham_data, vocab, total_ham_words, len(vocab))

print("P(w|Spam):", P_w_given_spam)
print("P(w|Ham):", P_w_given_ham)


P(w|Spam): {'suka': 0.07692307692307693, 'besar': 0.15384615384615385, 'beli': 0.15384615384615385, 'sekarang': 0.15384615384615385, 'rumah': 0.07692307692307693, 'di': 0.07692307692307693, 'saya': 0.07692307692307693, 'diskon': 0.15384615384615385, 'belajar': 0.07692307692307693}
P(w|Ham): {'suka': 0.125, 'besar': 0.0625, 'beli': 0.0625, 'sekarang': 0.125, 'rumah': 0.125, 'di': 0.125, 'saya': 0.125, 'diskon': 0.0625, 'belajar': 0.1875}



## **Langkah 6: Prediksi Kelas untuk Kalimat Baru**

### **Proses**:

Kita akan menggunakan rumus Naive Bayes untuk menghitung probabilitas kelas **Spam** dan **Ham** untuk kalimat baru `"sekarang belajar"`. Pilih kelas dengan probabilitas tertinggi.

In [6]:
import math

# Fungsi untuk menghitung P(C|x) menggunakan Naive Bayes
def predict(text, P_spam, P_ham, P_w_given_spam, P_w_given_ham, vocab):
    # Representasi BoW untuk teks
    text_bow = text_to_bow(text, vocab)
    
    # Hitung P(Spam | x) dan P(Ham | x)
    P_spam_x = math.log(P_spam)  # P(Spam)
    P_ham_x = math.log(P_ham)    # P(Ham)
    
    for word, count in zip(vocab, text_bow):
        if count > 0:
            P_spam_x += math.log(P_w_given_spam[word]) * count
            P_ham_x += math.log(P_w_given_ham[word]) * count
            
    return 'Spam' if P_spam_x > P_ham_x else 'Ham'

# Kalimat baru yang ingin diprediksi
new_text = ["sekarang", "belajar"]

# Prediksi kelasnya
prediction = predict(new_text, P_spam, P_ham, P_w_given_spam, P_w_given_ham, vocab)
print(f"Prediksi: {prediction}")


Prediksi: Ham



### **Kesimpulan**:

-   Kalimat baru `"sekarang belajar"` diklasifikasikan sebagai **Ham** (bukan spam).
    
-   Proses dimulai dengan **preprocessing**, lalu membuat **vocabulary**, dan menghitung **probabilitas kelas** dan **likelihood**.
    
-   **Naive Bayes** digunakan untuk menghitung probabilitas dan memilih kelas dengan probabilitas tertinggi